In [28]:
class Convolutional(object):
    def __init__(self, data, stride):
        if len(data.shape) != 4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        self.data = np.array(data)
        self.stride = stride
        self.data_length = data.shape[3]
        self.feat_num = data.shape[2]  # 9
        self.num, self.num_rev = self.generate_Num_and_ReversedNum(self.feat_num)
        self.conv_feat = len(self.num)
        self.step_list = self.generate_Step_List(self.data_length, self.stride)
        self.extracted_data = self.Main(self.data, self.feat_num, self.conv_feat, self.stride)
    def Main(self, data, feat_num, conv_feat, stride):
        print("------Start Extraction------")
        # multiprocessing
        num_cores = 5
        pool_convolution = mp.Pool(num_cores)
        t1 = time.time()
        result = {}
        for method in ["cov_and_corr"]:
#         for method in ["cov_and_corr","std","zscore","return","decay"]:
            result[method] = (pool_convolution.apply_async(self.Extraction, args=(data, feat_num, conv_feat, stride,method)))
        pool_convolution.close()
        pool_convolution.join()
        t2 = time.time()
        print("running time", int(t2 - t1))
        concat_list = []
        concat_list.append(result['cov_and_corr'][1])
        print(result['cov_and_corr'][0])
        print(result['cov_and_corr'][1])
        return feat_cat
    def Extraction(self, data, feat_num, conv_feat, stride,method):
        if method == "cov_and_corr":
            batch = nn.BatchNorm1d(conv_feat, affine=True)
            conv1 = self.ts_cov4d(self.data, self.stride, self.num, self.num_rev, self.step_list)
            conv2 = self.ts_corr4d(self.data, self.stride, self.num, self.num_rev, self.step_list, conv1)
            conv2 = torch.tanh(conv2)
            bc1 = batch(conv1.to(torch.float))
            bc2 = batch(conv2.to(torch.float))
            return bc1.detach().numpy() , bc2.detach().numpy()
        if method == "std":
            batch = nn.BatchNorm1d(feat_num, affine=True)
            conv3 = self.ts_stddev4d(self.data, self.stride, self.feat_num, self.step_list).to(torch.float)
            bc3 = batch(conv3).detach().numpy()
            return bc3
        if method == "zscore":
            batch = nn.BatchNorm1d(feat_num, affine=True)
            conv4 = self.ts_zscore(self.data, self.stride, self.feat_num, self.step_list).to(torch.float)
            bc4 = batch(conv4).detach().numpy()
            return bc4
        if method == "return":
            batch = nn.BatchNorm1d(feat_num, affine=True)
            conv5 = self.ts_return(self.data, self.stride, self.feat_num, self.step_list).to(torch.float)
            bc5 = batch(conv5)
            return bc5.detach().numpy()
        if method == "decay":
            batch = nn.BatchNorm1d(feat_num, affine=True)
            conv6 = self.ts_decaylinear(self.data, self.stride, self.feat_num, self.step_list).to(torch.float)
            bc6 = batch(conv6)
            return bc6.detach().numpy()
    def generateC(self, l1):
        if len(l1) == 1:
            return []
        v = [[l1[0], i] for i in l1[1:]]
        l1 = l1[1:]
        return v + self.generateC(l1)

    def generate_Num_and_ReversedNum(self, feat_nums):
        list1 = list(range(feat_nums))
        num = self.generateC(list1)
        num_rev = []
        for l in num:
            l1 = l.copy()
            l1.reverse()
            num_rev.append(l1)
        return num, num_rev

    def generate_Step_List(self, data_length, stride):
        # 11?¨2?3¤áD±í￡?è?1?êy?Y3¤?è2??ü??3y￡??òè?ê￡??3¤?è￡?è?1?ê￡??3¤?èD?óú5￡??òó?é?ò?2??áo?ò??e
        if data_length % stride == 0:
            step_list = list(range(0, data_length + stride, stride))
        elif data_length % stride <= 5:
            mod = data_length % stride
            step_list = list(range(0, data_length - stride, stride)) + [data_length]
        else:
            mod = data_length % stride
            step_list = list(range(0, data_length + stride - mod, stride)) + [data_length]
        return step_list

    """ Main Extraction"""

    def ts_cov4d(self, data, stride, num, num_rev, step_list):
        '''????4??êy?Yμ?D-·?2?'''
        '''data:[N,C,H,W],,W:price length,N:batch size'''
        l = []
        # ????μ?1y3ì?D??±?±￡3?keepdims=True
        for i in tqdm(range(len(step_list) - 1)):
            start = step_list[i]
            end = step_list[i + 1]
            sub_data1 = data[:, :, num, start:end]  # (2000, 1, 36, 2, 10)
            sub_data2 = data[:, :, num_rev, start:end]
            mean1 = sub_data1.mean(axis=4, keepdims=True)  # (2000, 1, 36, 2, 1)
            mean2 = sub_data2.mean(axis=4, keepdims=True)
            spread1 = sub_data1 - mean1  # (2000, 1, 36, 2, 10)
            spread2 = sub_data2 - mean2
            cov = ((spread1 * spread2).sum(axis=4, keepdims=True) / (sub_data1.shape[4] - 1)).mean(axis=3,
                                                                                                   keepdims=True)  # (2000, 1, 36, 1, 1)
            l.append(cov)
        corr = np.squeeze(np.array(l)).transpose(1, 2, 0).reshape(-1, self.conv_feat,
                                                                  len(step_list) - 1)  # (2000, 1, 36, 3)
        final = torch.from_numpy(corr)
        print("------Finished ts_cov4d----output shape: ", final.shape)
        return final

    def ts_corr4d(self, data, stride, num, num_rev, step_list, cov):
        '''????4??êy?Yμ??à1??μêy'''
        '''data:[N,C,H,W],,W:price length,N:batch size'''
        l = []
        for i in tqdm(range(len(step_list) - 1)):
            start = step_list[i]
            end = step_list[i + 1]
            sub_data1 = data[:, :, num, start:end]
            sub_data2 = data[:, :, num_rev, start:end]
            std1 = sub_data1.std(axis=4, keepdims=True)
            std2 = sub_data2.std(axis=4, keepdims=True)
            std = (std1 * std2).mean(axis=3, keepdims=True)
            del std1, std2  # êí·??ú′?
            l.append(std)
        std = np.squeeze(np.array(l)).transpose(1, 2, 0).reshape(-1, self.conv_feat, len(step_list) - 1)
        std[std == 0] = 1e-9
        fct = (sub_data1.shape[4] - 1) / sub_data1.shape[4]
        final = cov / torch.from_numpy(std) * fct
        del fct, std
        print("------Finished ts_corr4d----output shape: ", final.shape)
        return final

    def ts_stddev4d(self, data, stride, feat_num, step_list):
        if len(data.shape) != 4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in tqdm(range(len(step_list) - 1)):
            start = step_list[i]
            end = step_list[i + 1]
            sub_data1 = data[:, :, :, start:end]
            std1 = sub_data1.std(axis=3, keepdims=True)
            l.append(std1)
            del std1
        std = np.squeeze(np.array(l)).transpose(1, 2, 0).reshape(-1, feat_num, len(step_list) - 1)
        print("------Finished ts_stddev4d----output shape: ", torch.from_numpy(std).shape)
        return torch.from_numpy(std)

    def ts_zscore(self, data, stride, feat_num, step_list):
        if len(data.shape) != 4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in tqdm(range(len(step_list) - 1)):
            start = step_list[i]
            end = step_list[i + 1]
            sub_data1 = data[:, :, :, start:end]
            mean = sub_data1.mean(axis=3, keepdims=True)
            std = sub_data1.std(axis=3, keepdims=True)
            std[std == 0] = 1e-9
            z_score = mean / std
            l.append(z_score)
        z_score = np.squeeze(np.array(l)).transpose(1, 2, 0).reshape(-1, feat_num, len(step_list) - 1)
        #         z_score[z_score >= 6] = 6
        print("------Finished ts_zscore----output shape: ", torch.from_numpy(z_score).shape)
        return torch.from_numpy(z_score)

    def ts_return(self, data, stride, feat_num, step_list):
        if len(data.shape) != 4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        data[data == 0] = 1e-9
        l = []
        for i in tqdm(range(len(step_list) - 1)):
            start = step_list[i]
            end = step_list[i + 1]
            sub_data1 = data[:, :, :, start:end]
            ret = sub_data1[:, :, :, -1] / sub_data1[:, :, :, 0] - 1
            l.append(ret)
        z_data = np.squeeze(np.array(l)).transpose(1, 2, 0).reshape(-1, feat_num, len(step_list) - 1)
        z_data[z_data > 1] = 1
        print("------Finished ts_return----output shape: ", torch.from_numpy(z_data).shape)
        return torch.from_numpy(z_data)

    def ts_decaylinear(self, data, stride, feat_num, step_list):
        if len(data.shape) != 4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in tqdm(range(len(step_list) - 1)):
            start = step_list[i]
            end = step_list[i + 1]
            time_spread = end - start
            weight = np.arange(1, time_spread + 1)
            weight = weight / (weight.sum())
            sub_data1 = (data[:, :, :, start:end] * weight).mean(axis=3, keepdims=True)
            l.append(sub_data1)
        decay_data = np.squeeze(np.array(l)).transpose(1, 2, 0).reshape(-1, feat_num, len(step_list) - 1)
        final = torch.from_numpy(decay_data)
        print("------Finished ts_decaylinear----output shape: ", final.shape)
        return final

In [29]:
convolutional = Convolutional(trainx, 10)
feat_cat = convolutional.extracted_data

------Start Extraction------


100%|██████████| 3/3 [01:08<00:00, 22.89s/it]


------Finished ts_cov4d----output shape:  torch.Size([465192, 55, 3])


100%|██████████| 3/3 [01:13<00:00, 24.38s/it]


------Finished ts_corr4d----output shape:  torch.Size([465192, 55, 3])
running time 160


TypeError: 'ApplyResult' object is not subscriptable

NameError: name 'result' is not defined

In [8]:
def get_train_data(time_start, time_end):
    train_frame = dataframe_list[dataframe_list['timestamp'] < pd.to_datetime(str(time_start))]
    train_frame.set_index(["timestamp", "ticker"], inplace=True)

    # Train X and Train Y
    trainx, trainy = [], []
    trainx = np.array(train_frame.drop("target", axis=1))
    trainx = trainx.reshape(trainx.shape[0], 1, day, -1)  # x = (153,1,30,9)
    trainx = trainx.transpose(0, 1, 3, 2)  # x = (2555577, 1, 9, 30)
    trainy = np.array(train_frame['target']).reshape(-1, 1)  # x = (153,1,9,30)

    feat_num = trainx.shape[2]  # 9
    del train_frame
    print("trainx.shape: ", trainx.shape)
    print("trainy.shape: ", trainy.shape)
    return trainx, trainy, feat_num
trainx, trainy, feat_num = get_train_data("20160101", "20200630")

trainx.shape:  (465192, 1, 11, 30)
trainy.shape:  (465192, 1)
